# Tutorial RAG Azure
### 1. Criar um projeto no Azure AI Foundry - https://ai.azure.com/



### 2. Fazer deploy de modelos de embedding e completion (se já não estiverem disponíveis)



### 3. Criar um AI Search Service - https://portal.azure.com/#create/Microsoft.Search
 



#### 3.1 Criar um indíce e popular com documentos - create_search_index.py



### 4. Conectar o projeto Azure AI com AI Search - apenas pelo portal (ver o tutorial do git)





### Links uteis



https://github.com/MicrosoftDocs/azure-ai-docs/blob/main/articles/ai-studio/tutorials/copilot-sdk-create-resources.md





In [5]:
!python create_search_index2.py --index-name faq-index \
    --file-path assets/df_bacen.csv \
    --content-column answers \
    --fields questions,answers,categories \
    --vector-field questionVector


Starting index creation process...
Deleted existing index 'faq-index'


Traceback (most recent call last):
  File "C:\Users\giova\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\azure\search\documents\_search_client.py", line 655, in _index_documents_actions
    batch_response = self._client.documents.index(batch=batch, error_map=error_map, **kwargs)
  File "C:\Users\giova\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\azure\core\tracing\decorator.py", line 105, in wrapper_use_tracer
    return func(*args, **kwargs)
  File "C:\Users\giova\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\azure\search\documents\_generated\operations\_documents_operations.py", line 1200, in index
    map_error(status_code=response.status_code, response=response, error_map=error_map)
    ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [7]:
!python create_search_index_batch.py --index-name faq-index \
    --file-path assets/df_bacen.csv \
    --content-column answers \
    --fields questions,answers,categories \
    --vector-field contentVector


Starting index creation process...
Deleted existing index 'faq-index'
Uploaded batch 1: 100 documents successfully.
Uploaded batch 2: 100 documents successfully.
Uploaded batch 3: 100 documents successfully.
Uploaded batch 4: 100 documents successfully.
Uploaded batch 5: 100 documents successfully.
Uploaded batch 6: 100 documents successfully.
Uploaded batch 7: 100 documents successfully.
Uploaded batch 8: 100 documents successfully.
Uploaded batch 9: 100 documents successfully.
Uploaded batch 10: 100 documents successfully.
Uploaded batch 11: 100 documents successfully.
Uploaded batch 12: 100 documents successfully.
Uploaded batch 13: 100 documents successfully.
Uploaded batch 14: 5 documents successfully.
Index creation process completed successfully!


In [3]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

# Explicitly specify the tenant ID
credential = DefaultAzureCredential(
    tenant_id=""
)

# Use the credential in your project client
project = AIProjectClient.from_connection_string(
    conn_str=os.environ["AIPROJECT_CONNECTION_STRING"],
    credential=credential
)


TypeError: 'tenant_id' is not supported in DefaultAzureCredential.

In [4]:
!az login --tenant bb796b6d-2bb3-413f-915e-28bfa72da873


^C


In [1]:
### query_index.py: funcao generica para facilitar a pergunta
!python query_index.py --index-name faq-index \
    --query "Como faço uma reclamação sobre um banco?" \
    --vector-field contentVector \
    --select-fields id,questions,answers,categories \
    --top 5



In [12]:
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv


# Load environment variables
load_dotenv()

# Initialize logging


# Search connection
endpoint = os.getenv("SEARCH_ENDPOINT")
key = os.getenv("SEARCH_ADMIN_KEY")


def list_index_fields(index_name: str):
    from azure.search.documents.indexes import SearchIndexClient
    index_client = SearchIndexClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(key)
    )
    index = index_client.get_index(index_name)
    print("Fields in the index:")
    for field in index.fields:
        print(f"- {field.name} (type: {field.type})")
list_index_fields('faq-index')


Fields in the index:
- id (type: Edm.String)
- questions (type: Edm.String)
- answers (type: Edm.String)
- categories (type: Edm.String)
- contentVector (type: Collection(Edm.Single))


In [7]:
!python create_search_index2.py \
    --index-name stf-pdf-index \
    --file-path assets/stfacordo.pdf \
    --content-column content \
    --fields content,page_number \
    --vector-field contentVector





Starting index creation process...
Deleted existing index 'stf-pdf-index'
Uploaded 33 documents to 'stf-pdf-index' index
Index creation process completed successfully!


In [10]:
!python query_index.py --index-name stf-pdf-index \
    --query "Quem é o relator do processo?" \
    --vector-field contentVector \
    --select-fields content,page_number \
    --top 1

Querying index 'stf-pdf-index' with query: 'Quem é o relator do processo?'
Generating embedding for query: Quem é o relator do processo?
Performing vector search...
Result: {'content': 'Relatório\n27/05/2024 PRIMEIRA TURMA\nEMB.DECL. NO AG.REG. NOS SEGUNDOS EMB.DECL. NO RECURSO \nEXTRAORDINÁRIO  COM AGRAVO 1.412.405 PARANÁ\nRELATOR :MIN. FLÁVIO DINO\nEMBTE.(S):CAIO MACHADO DO COUTO COSTA \nADV.(A/S):TRACY JOSEPH REINALDET DOS SANTOS \nEMBDO.(A/S):MINISTÉRIO PÚBLICO FEDERAL \nPROC.(A/S)(ES):PROCURADOR-GERAL DA REPÚBLICA \nRELATÓRIO  \nO Senhor Ministro Flávio Dino (Relator):  Caio Machado do Couto  \nCosta  opõe  embargos  de  declaração  em  face  de  acórdão  de  agravo  \ninterno, por meio do qual mantida a decisão que negou seguimento ao  \nrecurso extraordinário com agravo.\nTranscrevo a ementa do acórdão embargado:\n“DIREITO  PENAL E  PROCESSUAL PENAL.  ACORDO  \nDE  NÃO  PERSECUÇÃO  PENAL  (ANPP).  ORIENTAÇÃO  \nJURISPRUDENCIAL  FIXADA  NA  PRIMEIRA  TURMA  DO  \nSUPREMO  TRIBUNA

In [ ]:
!python chat_with_products.py \
    --query "Quem é o relator do processo?" \
    --index-name stf-pdf-index \
    --vector-field contentVector \
    --select-fields id,content,page_number \
    --top 1



In [17]:
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv
import os

# Configure suas credenciais e endpoint
# Load environment variables
load_dotenv()

# Initialize logging


# Search connection
endpoint = os.getenv("SEARCH_ENDPOINT")
key = os.getenv("SEARCH_ADMIN_KEY")

index_client = SearchIndexClient(endpoint=endpoint, credential=AzureKeyCredential(key))

# Obtenha a definição do índice
index = index_client.get_index("stf-pdf-index")
print("Fields in the index:")
for field in index.fields:
    print(f"- {field.name} (type: {field.type})")


Fields in the index:
- id (type: Edm.String)
- content (type: Edm.String)
- page_number (type: Edm.Int32)
- contentVector (type: Collection(Edm.Single))


In [31]:
#Criar um search index
!python create_search_index2.py --index-name stf-pdf-index --file-path assets/stfacordo.pdf --content-column content --fields content,page_number --vector-field contentVector

#chat
!python chat_with_products.py --index-name stf-pdf-index --query "Quem é o relator do processo?" --vector-field contentVector --select-fields content,page_number --top 5

#opcional, query:
!python query_index.py --index-name stf-pdf-index --query "Quem é o relator do processo?" --vector-field contentVector --select-fields content,page_number --top 1

Performing vector search...
Retrieved 5 documents.
Chat response: ChatCompletionMessage(content='O relator do processo é o Ministro Flávio Dino.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
{'message': ChatCompletionMessage(content='O relator do processo é o Ministro Flávio Dino.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), 'context': {'thoughts': [{'title': 'Generated search query', 'description': 'N/A'}], 'grounding_data': [[{'content': 'Relatório\nARE 1412405 ED- SEGUNDOS-AGR-ED / PR \nANÁLISE  DA  LEGISLAÇÃO  INFRACONSTITUCIONAL  \nAPLICÁVEL E DO REEXAME DO CONJUNTO PROBATÓRIO  \nDOS  AUTOS.  INADMISSIBILIDADE.  SÚMULA  279/STF.  \nAGRAVO  NÃO  PROVIDO.  1.  No  julgamento  do  HC  \n233.147/SP ,  a  Primeira  Turma  do  Supremo  Tribunal  Federal  \nfixou  orientação  no  sentido  de  que,  até  a  conclusão,  pelo  \nTribunal Pleno, da análise do HC 185.913/DF, ‘nas ações penais  \niniciadas antes da ent

In [13]:

!python create_search_index2.py --index-name bacen_index \
    --file-path assets/df_bacen.csv \
    --content-column answers \
    --fields questions,answers,categories \
    --vector-field contentVector


Starting index creation process...

Traceback (most recent call last):
  File "C:\Users\giova\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\azure\search\documents\_search_client.py", line 655, in _index_documents_actions
    batch_response = self._client.documents.index(batch=batch, error_map=error_map, **kwargs)
  File "C:\Users\giova\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\azure\core\tracing\decorator.py", line 105, in wrapper_use_tracer
    return func(*args, **kwargs)
  File "C:\Users\giova\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\azure\search\documents\_generated\operations\_documents_operations.py", line 1200, in index
    map_error(status_code=response.status_code, response=response, error_map=error_map)
    ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


Deleted existing index 'bacen_index'


In [1]:
    
!python chat_with_products.py --index-name faq-index \
    --query "O que são moedas virtuais?" \
    --vector-field contentVector \
    --select-fields answers \
    --top 20

Traceback (most recent call last):
  File "d:\ragazure-20250115T085251Z-001 - Copia\ragazure\chat_with_products.py", line 3, in <module>
    from opentelemetry import trace
ModuleNotFoundError: No module named 'opentelemetry'


In [26]:

!python evaluate.py --dataset-path assets/chat_eval_data_fixed.jsonl --evaluation-name evaluate_chat_with_products --output-path ./myevalresults.json


2025-01-09 15:09:53,951 [INFO] No environment configuration found.
2025-01-09 15:09:53,951 [INFO] ManagedIdentityCredential will use IMDS
Traceback (most recent call last):
  File "f:\woopi\ragazure\evaluate.py", line 127, in <module>
    main()
    ~~~~^^
  File "f:\woopi\ragazure\evaluate.py", line 87, in main
    "azure_endpoint": connection.endpoint_url,
                      ^^^^^^^^^^
NameError: name 'connection' is not defined. Did you mean: 'ConnectionType'?
